In [ ]:
from src.mazeData import *
from src.maze2025GraphClass import mazeGraph
from pyvis.network import Network
from src.PS_agentPrograms import *
from src.mazeProblemClass import MazeProblem
from src.agents import *
from src.naigationEnvironmentClass import MazeNavigationEnvironment
from src.Lab5Env import *

In [ ]:
def makeMaze(n):
  size = (n,n)
  proba_0 =0.20 # resulting array will have 25%? of zeros
  proba_Food =0.1 # resulting array will have 10%? of enemy
  arrMaze=np.random.choice([0, 1, 2], size=size, p=[proba_0, 1-proba_0-proba_Food, proba_Food] )

  return arrMaze

initState = (random.randint(0,9),random.randint(0,9))
goalState = (random.randint(0,9),random.randint(0,9))

Pinky = (random.randint(0,9),random.randint(0,9))
Inky = (random.randint(0,9),random.randint(0,9))
Blinky = (random.randint(0,9),random.randint(0,9))
Clyde = (random.randint(0,9),random.randint(0,9))
Betrayus = (random.randint(0,9),random.randint(0,9))

goals = [goalState]

#im thinking of doing something close to
#for each node in maze of [x][y].node == 2:
#  goals.append(node)
#this way we can add the food nodes as they are added/all at once. not sure how else we would do this

mazeSize=10
mainMaze = makeMaze(mazeSize)
MazeCheck(mainMaze,initState,goalState)
GhostCheck(mainMaze,Pinky,Inky,Blinky,Clyde,Betrayus)
mazeAvalActs=defineMazeAvailableActions(mainMaze)
maze1TM=makeMazeTransformationModel(mazeAvalActs)
mazeWorldGraph=mazeGraph(maze1TM, mazeStatesLocations(list(maze1TM.keys())))

maze1TM

In [ ]:
mainMaze

In [ ]:
maze1TM

In [ ]:
mazeAvalActs

In [ ]:
net_maze = Network(heading="Lab 4 Maze",
                bgcolor ="#242020",
                font_color = "white",
                height = "750px",
                width = "100%" 
)
nodeColors={
    "wall":"Blue",
    "path": "White",
    "Goal": "Green",
    "Start": "Red",
    "Enemy": "Orange",
    "Food": "Yellow"
}
nodeColorsList=[]

for node in mazeWorldGraph.origin.keys():
    if mainMaze[node[0],node[1]]==1:
        nodeColorsList.append(nodeColors["path"])
    elif mainMaze[node[0],node[1]]==0:
        nodeColorsList.append(nodeColors["wall"])
    elif mainMaze[node[0],node[1]]==3:
        nodeColorsList.append(nodeColors["Enemy"])
    else:
        nodeColorsList.append(nodeColors["Food"])

nodes=["-".join(str(item) for item in el) for el in mazeWorldGraph.origin.keys()]

x_coords = []
y_coords = []

for node in mazeWorldGraph.origin.keys():
    x,y=mazeWorldGraph.getLocation(node)
    x_coords.append(x)
    y_coords.append(y)

sizes=[10]*len(nodes)
net_maze.add_nodes(nodes, color=nodeColorsList, x=x_coords, y=y_coords, size=sizes, title=nodes)

for node in net_maze.nodes:
    node['label']=''

edge_weights = {(intTupleTostr(k), intTupleTostr(v2)) : k2 for k, v in mazeWorldGraph.origin.items() for k2, v2 in v.items()}

edges=[]

for node_source in mazeWorldGraph.nodes():
    for node_target, action in mazeWorldGraph.get(node_source).items():
        #node_target or node_source is a tuple -> convert to str
        if (intTupleTostr(node_source),intTupleTostr(node_target)) not in edges and (intTupleTostr(node_target), intTupleTostr(node_source)):
            net_maze.add_edge(intTupleTostr(node_source),intTupleTostr(node_target), label=edge_weights[(intTupleTostr(node_source),intTupleTostr(node_target))])
            edges.append((intTupleTostr(node_source),intTupleTostr(node_target)))

for node in net_maze.nodes:
    if node['id']==intTupleTostr(goalState):
        node['color']=nodeColors['Goal']
    elif node['id']==intTupleTostr(initState):
        node['color']=nodeColors['Start']

In [ ]:
net_maze.toggle_physics(False)

In [ ]:
mp1=MazeProblem(initState, goalState, mazeWorldGraph)

In [ ]:
AstarAgent=ProblemSolvingMazeAgentAstar(initState, mazeWorldGraph, goalState)
IDA_starAgent=IDA_StarSearchAgent(initState, mazeWorldGraph, goalState)

In [ ]:
maze_Env1=Lab5NavEnvironment(mazeWorldGraph, mainMaze)
maze_Env2=Lab5NavEnvironment(mazeWorldGraph, mainMaze)

#maze_Env1=MazeNavigationEnvironment(mazeWorldGraph)
#maze_Env2=MazeNavigationEnvironment(mazeWorldGraph)

In [ ]:
maze_Env1.add_thing(AstarAgent)

In [ ]:
maze_Env1.run()

In [ ]:
maze_Env2.add_thing(IDA_starAgent)

In [ ]:
maze_Env2.run()

In [ ]:
net_maze.show("graphMaze1.html", notebook=False)